In [ ]:
import sys
import warnings, tqdm

warnings.filterwarnings("ignore", category=tqdm.TqdmWarning)
sys.modules['tqdm.notebook'] = tqdm
sys.modules['tqdm.autonotebook'] = tqdm

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    import os
    
    # Mount Google Drive first
    print("📁 Mounting Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Verify your dataset folder exists
    drive_dataset_path = "/content/drive/MyDrive/datasets/IRMAS"
    if os.path.exists(drive_dataset_path):
        print(f"✅ Found dataset folder at: {drive_dataset_path}")
    else:
        print(f"⚠️  Dataset folder not found at: {drive_dataset_path}")
        print("Creating the folder structure...")
        os.makedirs(drive_dataset_path, exist_ok=True)

    # Always start fresh and clone the specific branch
    print("🗑️ Cleaning up any existing project...")
    %cd /content
    !rm -rf DL_Project

    #TODO: Fix the branch according to the latest changes
    print("📥 Cloning specific branch 'master'...")
    !git clone -b master https://github.com/ofekdd/DL_Project.git
    %cd DL_Project

    # Verify we're on the correct branch
    print("🔍 Verifying branch...")
    !git branch
    !git log --oneline -n 3

    # Install dependencies
    print("📦 Installing dependencies...")
    !pip install -r requirements.txt

    print("✅ Setup complete with branch 'master'!")

In [27]:
# Check the current working directory and ensure it is the project root
from pathlib import Path
print("CWD :", Path.cwd())                    # where the kernel is running
print("Exists?", Path('configs').is_dir())    # should be True if CWD is project root


CWD : /home/odahan/Technion/Semester_8/Deep_Learning/Project/notebooks
Exists? False


In [28]:
import yaml
import os

# Define the path to the YAML configuration file
workspace = '/home/odahan/Technion/Semester_8/Deep_Learning/Project'
yaml_path = 'configs/panns_enhanced.yaml' if IN_COLAB else f'{workspace}/configs/panns_enhanced.yaml'
print(yaml_path)
# Open and load the YAML file
with open(yaml_path, 'r') as file:
    cfg = yaml.safe_load(file)

print("PANNs-enhanced configuration:")
for key, value in cfg.items():
    print(f"  {key}: {value}")

/home/odahan/Technion/Semester_8/Deep_Learning/Project/configs/multi_stft_cnn.yaml
9cnn configuration:
  model_name: multi_stft_cnn
  sample_rate: 22050
  n_mels: 64
  hop_length: 512
  batch_size: 8
  num_epochs: 50
  learning_rate: 2e-4
  num_workers: 4
  n_branches: 9
  branch_output_dim: 128


In [ ]:
# Download and setup the IRMAS dataset using Google Drive
from data.download_irmas import main as download_irmas_main
import pathlib
import os
import zipfile

# Set DATA_CACHE to your Google Drive folder
if IN_COLAB:
    DATA_CACHE = "/content/drive/MyDrive/datasets/IRMAS"
    print(f"📁 Using Google Drive dataset folder: {DATA_CACHE}")
else:
    # Check for dataset in home directory (for local development)
    home_dataset_path = pathlib.Path.home() / "datasets" / "irmas"
    DATA_CACHE = str(home_dataset_path if home_dataset_path.exists() else "data/raw")

# Create the dataset directory if it doesn't exist
os.makedirs(DATA_CACHE, exist_ok=True)

# Check what we have in the Google Drive folder
print("🔍 Current contents of your Google Drive dataset folder:")
if os.path.exists(DATA_CACHE):
    items = os.listdir(DATA_CACHE)
    for item in items:
        item_path = os.path.join(DATA_CACHE, item)
        if os.path.isdir(item_path):
            print(f"  📁 {item}/")
        else:
            print(f"  📄 {item}")
    if not items:
        print("  (empty)")
else:
    print("  Folder does not exist, will be created")

# Define all IRMAS dataset parts
irmas_datasets = {
    "IRMAS-TrainingData.zip": "IRMAS-TrainingData",
    "IRMAS-TestingData-Part1.zip": "IRMAS-TestingData-Part1", 
    "IRMAS-TestingData-Part2.zip": "IRMAS-TestingData-Part2",
    "IRMAS-TestingData-Part3.zip": "IRMAS-TestingData-Part3"
}

# Check which parts are missing
missing_zips = []
missing_folders = []

for zip_name, folder_name in irmas_datasets.items():
    zip_path = pathlib.Path(DATA_CACHE) / zip_name
    folder_path = pathlib.Path(DATA_CACHE) / folder_name
    
    if not zip_path.exists():
        missing_zips.append(zip_name)
        print(f"❌ Missing: {zip_name}")
    else:
        print(f"✅ Found: {zip_name}")
    
    if not folder_path.exists():
        missing_folders.append(folder_name)
        print(f"❌ Missing extracted: {folder_name}")
    else:
        print(f"✅ Found extracted: {folder_name}")

# Download missing zip files
if missing_zips:
    print(f"\n📥 Downloading {len(missing_zips)} missing dataset(s) to Google Drive...")
    download_irmas_main(pathlib.Path(DATA_CACHE))
else:
    print("\n✅ All IRMAS zip files already present in Google Drive. Skipping download.")

# Extract missing folders
for zip_name, folder_name in irmas_datasets.items():
    zip_path = pathlib.Path(DATA_CACHE) / zip_name
    folder_path = pathlib.Path(DATA_CACHE) / folder_name
    
    if zip_path.exists() and not folder_path.exists():
        print(f"📦 Extracting {zip_name}...")
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(DATA_CACHE)
            print(f"✅ Extracted {folder_name}")
        except Exception as e:
            print(f"❌ Failed to extract {zip_name}: {e}")

# Final verification
print("\n🔍 Final verification of dataset extraction:")
all_ready = True
for folder_name in irmas_datasets.values():
    expected_path = pathlib.Path(DATA_CACHE) / folder_name
    if expected_path.exists():
        print(f"✅ {folder_name} found at {expected_path}")
    else:
        print(f"❌ {folder_name} missing at {expected_path}")
        all_ready = False

if all_ready:
    print("🎉 All IRMAS dataset parts are ready in Google Drive!")
    # Set IRMAS root to base path for compatibility with existing code
    irmas_root = pathlib.Path(DATA_CACHE)
    print(f"📂 IRMAS base path set to: {irmas_root}")
else:
    print("\n⚠️ Some dataset parts are missing. Please check for errors above.")
    irmas_root = None

# Show final folder structure
print(f"\n📊 Final Google Drive dataset structure:")
if os.path.exists(DATA_CACHE):
    for item in sorted(os.listdir(DATA_CACHE)):
        item_path = os.path.join(DATA_CACHE, item)
        if os.path.isdir(item_path):
            # Count files in each subfolder
            try:
                file_count = len([f for f in os.listdir(item_path) if os.path.isfile(os.path.join(item_path, f))])
                print(f"  📁 {item}/ ({file_count} files)")
            except:
                print(f"  📁 {item}/")
        else:
            # Show file size for zip files
            try:
                size_mb = os.path.getsize(item_path) / (1024*1024)
                print(f"  📄 {item} ({size_mb:.1f} MB)")
            except:
                print(f"  📄 {item}")

In [ ]:
# Fix NumPy compatibility issue
import sys

print("🔧 Fixing NumPy compatibility...")

# Check current NumPy version
import numpy as np

print(f"Current NumPy version: {np.__version__}")

# If NumPy 2.0+, we need to downgrade or use a workaround
if int(np.__version__.split('.')[0]) >= 2:
    print("⚠️  NumPy 2.0+ detected. Installing compatible version...")
    !pip install "numpy<2.0" --quiet

    # Restart the kernel to load the new NumPy version
    print("🔄 Restarting kernel to load compatible NumPy...")
    import os

    os.kill(os.getpid(), 9)  # This will restart the kernel in Colab
else:
    print("✅ NumPy version is compatible")

In [ ]:
from data.download_irmas import load_irmas_audio_dataset, load_irmas_testing_dataset
import pathlib

if irmas_root and irmas_root.exists():

    print("📁 Dataset creation settings from config:")
    print(f"   max_original_samples: {cfg.get('max_original_samples', 50)}")
    print(f"   num_mixtures: {cfg.get('num_mixtures', 100)}")
    print(f"   min_instruments: {cfg.get('min_instruments', 1)}")
    print(f"   max_instruments: {cfg.get('max_instruments', 2)}")

    # Normalize root
    base_root = irmas_root.parent if irmas_root.name == "IRMAS-TrainingData" else irmas_root

    # Define separate paths
    training_path = base_root / "IRMAS-TrainingData"
    testing_paths = [base_root / f"IRMAS-TestingData-Part{i}" for i in range(1, 4)]

    print(f"📂 IRMAS paths:")
    print(f"   ├─ Training: {training_path}")
    for tp in testing_paths:
        print(f"   └─ Test Part: {tp}")

    # Load training data (single-label)
    original_dataset = load_irmas_audio_dataset(base_root, cfg,
                                            max_samples=cfg.get("max_original_samples"))

    # Load test data (multi-label)
    test_datasets = []
    for _ in testing_paths:        # paths 1-3 are *ignored* here
        test_datasets.extend(load_irmas_testing_dataset(base_root, cfg))

    # Merge datasets if needed
    total_loaded = len(original_dataset) + len(test_datasets)
    print(f"\n📊 Final dataset summary:")
    print(f"   ✅ Training samples loaded: {len(original_dataset)}")
    print(f"   ✅ Testing samples loaded: {len(test_datasets)}")
    print(f"   ✅ Total samples loaded:   {total_loaded}")

else:
    print("❌ IRMAS root not found or invalid. Please run the download step first.")

In [ ]:
if irmas_root:
    print(f"IRMAS dataset found at: {irmas_root}")
    
    # Use Google Drive for processed features cache as well to avoid reprocessing
    if IN_COLAB:
        PROCESSED_DIR = "/content/drive/MyDrive/datasets/IRMAS_processed_features"
        print(f"📁 Using Google Drive for processed features: {PROCESSED_DIR}")
    else:
        PROCESSED_DIR = "data/processed"

    # Create processed directory if it doesn't exist
    os.makedirs(PROCESSED_DIR, exist_ok=True)

    # Use config value for original data percentage
    original_data_percentage = cfg.get('original_data_percentage', 0.1)
    print(f"Using {original_data_percentage*100}% of original IRMAS data (from config)")

    # Check if processed features already exist
    expected_processed_dirs = [
        os.path.join(PROCESSED_DIR, "train"),
        os.path.join(PROCESSED_DIR, "val"), 
        os.path.join(PROCESSED_DIR, "test")
    ]
    
    processing_needed = not all(os.path.exists(d) and os.listdir(d) for d in expected_processed_dirs)
    
    if processing_needed:
        print("📊 Processed features not found or incomplete. Starting preprocessing...")
        from data.preprocess import preprocess_data
        
        preprocess_data(
            irmas_root=irmas_root,
            out_dir=PROCESSED_DIR,
            cfg=cfg,
            original_data_percentage=original_data_percentage
        )
        print(f"✅ Preprocessing complete! Features saved to Google Drive: {PROCESSED_DIR}")
    else:
        print("✅ Processed features found in Google Drive. Skipping preprocessing.")
        
    # Show processed features summary
    print(f"\n📊 Processed features summary:")
    for split in ["train", "val", "test"]:
        split_path = os.path.join(PROCESSED_DIR, split)
        if os.path.exists(split_path):
            file_count = len([f for f in os.listdir(split_path) if os.path.isdir(os.path.join(split_path, f))])
            print(f"  📁 {split}/ ({file_count} sample folders)")
        else:
            print(f"  ❌ {split}/ (missing)")

else:
    print("Could not locate IRMAS dataset after download. Check paths and try again.")

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 📦  Configure paths & echo training settings
# ──────────────────────────────────────────────────────────────────────────────
print("🔧 Configuring data paths and training settings...")

# 1) show YAML-driven parameters
base_max_samples = cfg.get("max_samples", None)
print("📁 Base configuration from YAML:")
print(f"   max_samples            : {base_max_samples}")
print(f"   max_original_samples   : {cfg.get('max_original_samples', 50)}")
print(f"   num_mixtures           : {cfg.get('num_mixtures', 100)}")
print(f"   min_instruments        : {cfg.get('min_instruments', 1)}")
print(f"   max_instruments        : {cfg.get('max_instruments', 2)}")
print(f"   original_data_percentage : {cfg.get('original_data_percentage', 0.1)}")

# 2) notebook override notice
if base_max_samples != cfg.get("max_samples"):
    print(f"⚠️  Notebook override: max_samples changed to {cfg.get('max_samples')}")

# 3) define processed-data locations (use Google Drive path)
if IN_COLAB:
    PROCESSED_DIR = "/content/drive/MyDrive/datasets/IRMAS_processed_features"
else:
    PROCESSED_DIR = "data/processed"
    
cfg.update(
    {
        "data_dir": PROCESSED_DIR,
        "train_dir": f"{PROCESSED_DIR}/train",
        "val_dir": f"{PROCESSED_DIR}/val",
        "test_dir": f"{PROCESSED_DIR}/test",
    }
)

print("\n📂 Data directories:")
print(f"   Processed data : {cfg['data_dir']}")
print(f"   Training       : {cfg['train_dir']}")
print(f"   Validation     : {cfg['val_dir']}")
print(f"   Test           : {cfg['test_dir']}")

# ──────────────────────────────────────────────────────────────────────────────
# 🔍  Verify directory existence & detailed sample counts
# ──────────────────────────────────────────────────────────────────────────────
import pathlib

def count_sample_folders(split_path: pathlib.Path):
    counts = dict(original=0, mixed=0, irmasTest=0, other=0)
    if not split_path.exists():
        return counts  # all zeros
    for d in split_path.iterdir():
        if not d.is_dir():
            continue
        n = d.name
        if n.startswith("original_"):
            counts["original"] += 1
        elif n.startswith("mixed_"):
            counts["mixed"] += 1
        elif n.startswith("irmasTest_"):
            counts["irmasTest"] += 1
        else:
            counts["other"] += 1
    return counts


print("\n🔍 Verifying data directories & split composition:")
for split in ["train", "val", "test"]:
    path = pathlib.Path(cfg[f"{split}_dir"])
    counts = count_sample_folders(path)
    total = sum(counts.values())

    status = "✅" if total else "❌"
    print(f"\n{status} {split.upper()} ({path}): {total} total sample folders")
    print(f"      • original_:  {counts['original']}")
    print(f"      • mixed_:     {counts['mixed']}")
    print(f"      • irmasTest_: {counts['irmasTest']}")
    if counts["other"]:
        print(f"      • other:      {counts['other']}  ← check if expected!")

    # sanity warnings
    if split in ("train", "val") and counts["irmasTest"]:
        print("      ⚠️  Unexpected irmasTest_ folders in this split!")
    if split == "test" and (counts["original"] or counts["mixed"]):
        print("      ⚠️  Test split should contain ONLY irmasTest_ folders.")

# ──────────────────────────────────────────────────────────────────────────────
# 🎛️  Final training configuration summary
# ──────────────────────────────────────────────────────────────────────────────
print("\n✅ Final training configuration:")
print(f"   Training samples limit : {cfg.get('max_samples', 'unlimited')}")
print(f"   Batch size             : {cfg.get('batch_size')}")
print(
    f"   Validation limit       : {cfg.get('limit_val_batches', 1.0)} "
    f"({'percentage' if cfg.get('limit_val_batches', 1.0) <= 1 else 'batches'})"
)
print(f"   Learning rate          : {cfg.get('learning_rate')}")
print(f"   Epochs                 : {cfg.get('num_epochs')}")

In [ ]:
# Import required modules for the model
import torch
from var import LABELS
from models.Conv_wavelet_cnn import WaveletCNN  # <- your new file

n_classes = len(LABELS)
sr = cfg["sample_rate"]

# Create the Wavelet-based model
model = WaveletCNN(n_classes=n_classes, sr=sr)
print(model)

# Optional: torchinfo summary
try:
    from torchinfo import summary
    # pretend 2 seconds of audio
    T = int(sr * 2.0)
    summary(model, input_size=(1, T))
except Exception as e:
    print(f"(torchinfo unavailable or failed: {e})")

# quick smoke test on CPU (or GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device).eval()

with torch.no_grad():
    x = torch.zeros(2, int(sr * 2.0), device=device)  # [B, T]
    y = model(x)
print("Smoke test OK:", y.shape)




In [ ]:
# %%
# TRAINING LAUNCH (Wavelet version)
from training.train import main as train_main  # make sure this uses WaveletCNN inside


print("📦 Training config summary:")
print(f"  train_dir: {cfg.get('train_dir')}")
print(f"  val_dir  : {cfg.get('val_dir')}")
print(f"  batch_size: {cfg.get('batch_size')}")
print(f"  lr        : {cfg.get('learning_rate')}")
print(f"  epochs    : {cfg.get('num_epochs')}")

try:
    train_main(cfg)
    print("🏁 Training completed successfully!")
except Exception as err:
    import traceback
    print(f"❌ Training error: {err}")
    traceback.print_exc(limit=2)


In [ ]:
# Inference and visualization using the test set


In [ ]:
# %%
# Inference (wavelet model, raw waveforms)
import glob, re, yaml, sys, traceback
from pathlib import Path
import numpy as np
import librosa

def find_best_checkpoint(log_dir="lightning_logs"):
    """
    Return the .ckpt with the highest val_mAP (matches your ModelCheckpoint filename).
    """
    ckpts = glob.glob(f"{log_dir}/*/checkpoints/*.ckpt")
    if not ckpts:
        print(f"❌ No checkpoints found under {log_dir}")
        return None

    print(f"🔍 Found {len(ckpts)} candidate checkpoint(s)")

    best, best_map, best_epoch = None, -1.0, -1
    pat_epoch = re.compile(r"epoch=(\d+)")
    pat_map   = re.compile(r"val_mAP=([0-9]+(?:\.[0-9]*)?)")  # <-- NOTE: mAP key

    for c in ckpts:
        m_ep  = pat_epoch.search(c)
        m_map = pat_map.search(c)
        if not (m_ep and m_map):
            continue
        epoch = int(m_ep.group(1))
        vmap  = float(m_map.group(1).rstrip("."))
        if vmap > best_map or (vmap == best_map and epoch > best_epoch):
            best, best_map, best_epoch = c, vmap, epoch

    if best:
        print(f"✅ Best checkpoint → {best} | epoch {best_epoch} | val_mAP {best_map:.4f}")
    return best or ckpts[0]

ckpt_path = find_best_checkpoint()
if ckpt_path is None:
    sys.exit("🛑  Aborting – no checkpoints found.")

# Reload YAML
with open(yaml_path) as fh:
    cfg = yaml.safe_load(fh)

# Rebuild wavelet model and load weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = WaveletCNN(n_classes=len(LABELS), sr=cfg["sample_rate"]).to(device).eval()

state = torch.load(ckpt_path, map_location="cpu")
state_dict = state.get("state_dict", state)
# strip 'model.' prefix if present
cleaned = {}
for k, v in state_dict.items():
    cleaned[k[6:]] = v if k.startswith("model.") else v

missing, unexpected = model.load_state_dict(cleaned, strict=False)
if missing:   print("Missing keys:", missing[:5], "…")  # usually harmless if layer names changed slightly
if unexpected: print("Unexpected keys:", unexpected[:5], "…")

def predict_waveform(model, wav_path, cfg, device):
    y, sr = librosa.load(wav_path, sr=cfg["sample_rate"], mono=True)
    x = torch.from_numpy(y.astype(np.float32)).to(device).unsqueeze(0)  # [1, T]
    with torch.no_grad():
        probs = model(x).squeeze(0).cpu().numpy()  # already sigmoid in model
    return {label: float(probs[i]) for i, label in enumerate(LABELS)}

# Gather test wavs
MAX_TEST_SAMPLES = cfg.get("max_test_samples", 5)
wav_candidates = []
tdir = cfg.get("test_dir")
if tdir and Path(tdir).exists():
    wav_candidates = list(Path(tdir).rglob("*.wav"))
if not wav_candidates and "irmas_root" in globals() and irmas_root:
    wav_candidates = list(Path(irmas_root).rglob("*.wav"))
wav_candidates = [p for p in wav_candidates if "IRMAS-TestingData-Part" in str(p)]
if MAX_TEST_SAMPLES and len(wav_candidates) > MAX_TEST_SAMPLES:
    wav_candidates = wav_candidates[:MAX_TEST_SAMPLES]

print(f"🗂️  Running inference on {len(wav_candidates)} file(s)")

# Simple single-label accuracy (top-1) if GT is in file name
import re
irmas_to_label = {
    'cel': 'cello','cla':'clarinet','flu':'flute',
    'gac':'acoustic_guitar','gel':'acoustic_guitar',
    'org':'organ','pia':'piano','sax':'saxophone',
    'tru':'trumpet','vio':'violin','voi':'voice'
}
def parse_gt_from_name(name: str):
    codes = re.findall(r"\[([a-z]{3})\]", name)
    return [irmas_to_label[c] for c in codes if c in irmas_to_label]

correct = 0
total   = 0
for p in wav_candidates:
    preds = predict_waveform(model, str(p), cfg, device)
    top = max(preds.items(), key=lambda kv: kv[1])[0]
    gt = parse_gt_from_name(p.name)
    if gt:
        total += 1
        if top in gt:
            correct += 1
    print(f"{p.name:45s} → top: {top:15s} | gt: {gt}")

if total:
    print(f"\n🎯 Single-label top-1 accuracy: {correct}/{total} = {correct/total:.1%}")
else:
    print("\n(no ground-truth tags found in filenames; skipped accuracy)")
